# Deploying Campaigns and Filters<a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [3]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd

In [4]:
%store -r

In [5]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Create campaigns <a class="anchor" id="create"></a>
[Back to top](#top)

A campaign is a hosted solution version; an endpoint which you can query for recommendations. Pricing is set by estimating throughput capacity (requests from users for personalization per second). When deploying a campaign, you set a minimum throughput per second (TPS) value. This service, like many within AWS, will automatically scale based on demand, but if latency is critical, you may want to provision ahead for larger demand. For this POC and demo, all minimum throughput thresholds are set to 1. For more information, see the [pricing page](https://aws.amazon.com/personalize/pricing/).

Let's start deploying the campaigns.

### User Personalization

Deploy a campaign for your User Personalization solution version. It can take around 10 minutes to deploy a campaign. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create multiple campaigns. So we will set up the while loop for all of the campaigns further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

You now have the option of setting additional configuration for the campaign, which allows you to adjust the exploration Amazon Personalize does for the item recommendations and therefore adjust the results. These settings are only available if you’re creating a campaign whose solution version uses the user-personalization recipe. The configuration options are as follows:

    explorationWeight – Higher values for explorationWeight signify higher exploration; new items with low impressions are more likely to be recommended. A value of 0 signifies that there is no exploration and results are ranked according to relevance. You can set this parameter in a range of [0,1] and its default value is 0.3.
    
    explorationItemAgeCutoff – This is the maximum duration in days relative to the latest interaction(event) timestamp in the training data. For example, if you set explorationItemAgeCutoff to 7, the items with an age over or equal to 7 days aren’t considered cold items and there is no exploration on these items. You may still see some items older than or equal to 7 days in the recommendation list because they’re relevant to the user’s interests and are of good quality even without the help of the exploration. The default value for this parameter is 30, and you can set it to any value over 0.
    
We will use the explorationWeight in the "06_Adding_New_Items_and_Updating_Model.ipynb", so we will set the explorationWeight to 0.9, which will cause the new (cold start) items to be recommended more often.

In [9]:
userpersonalization_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-userpersonalization",
    solutionVersionArn = userpersonalization_solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig={
        'itemExplorationConfig': {
            'explorationWeight': '0.9'
        }
    }
)

userpersonalization_campaign_arn = userpersonalization_create_campaign_response['campaignArn']
print(json.dumps(userpersonalization_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:832194813872:campaign/personalize-poc-userpersonalization",
  "ResponseMetadata": {
    "RequestId": "f42466d2-c57f-4aba-ab67-42dca9ffa8ff",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 02 Apr 2021 01:18:14 GMT",
      "x-amzn-requestid": "f42466d2-c57f-4aba-ab67-42dca9ffa8ff",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [4]:
userpersonalization_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-userpersonalization",
    solutionVersionArn = userpersonalization_solution_version_arn,
    minProvisionedTPS = 1
)

userpersonalization_campaign_arn = userpersonalization_create_campaign_response['campaignArn']
print(json.dumps(userpersonalization_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:832194813872:campaign/personalize-poc-userpersonalization",
  "ResponseMetadata": {
    "RequestId": "bf5b946b-8671-4d7a-a280-335826c52320",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 02 Feb 2021 00:00:27 GMT",
      "x-amzn-requestid": "bf5b946b-8671-4d7a-a280-335826c52320",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS

Deploy a campaign for your SIMS solution version. It can take around 10 minutes to deploy a campaign. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create multiple campaigns. So we will set up the while loop for all of the campaigns further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [10]:
sims_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-SIMS",
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = sims_create_campaign_response['campaignArn']
print(json.dumps(sims_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:832194813872:campaign/personalize-poc-SIMS",
  "ResponseMetadata": {
    "RequestId": "8ff69f57-ef4f-40b2-9072-fd2770525584",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 02 Apr 2021 01:18:48 GMT",
      "x-amzn-requestid": "8ff69f57-ef4f-40b2-9072-fd2770525584",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Personalized Ranking

Deploy a campaign for your personalized ranking solution version. It can take around 10 minutes to deploy a campaign. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create multiple campaigns. So we will set up the while loop for all of the campaigns further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [11]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:832194813872:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "c105b65b-fc5d-45df-be92-138d58fb6e7f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 02 Apr 2021 01:18:51 GMT",
      "x-amzn-requestid": "c105b65b-fc5d-45df-be92-138d58fb6e7f",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### View campaign creation status

As promised, how to view the status updates in the console:

* In another browser tab you should already have the AWS Console up from opening this notebook instance. 
* Switch to that tab and search at the top for the service `Personalize`, then go to that service page. 
* Click `View dataset groups`.
* Click the name of your dataset group, most likely something with POC in the name.
* Click `Campaigns`.
* You will now see a list of all of the campaigns you created above, including a column with the status of the campaign. Once it is `Active`, your campaign is ready to be queried.

Or simply run the cell below to keep track of the campaign creation status.

In [12]:
in_progress_campaigns = [
    userpersonalization_campaign_arn,
    sims_campaign_arn,
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:832194813872:campaign/personalize-poc-userpersonalization
At least one campaign build is still in progress
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:832194813872:campaign/personalize-poc-SIMS
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:832194813872:campaign/personalize-poc-rerank


## Create Static Filters <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can create filters. Filters can be created for both Items and Events. Filters can also be created dynamically for "IN" and "=" operation, which is covered in the 05_Interacting_with_Campaigns_and_Filters notebook. For range queries, you need to continue to use static filters. 
Range queries use the following operations: NOT IN, <, >, <=, and >=.

A few common use cases for static filters in Video On Demand are:

Categorical filters based on Item Metadata (that are range based) - Often your item metadata will have information about the title such as year, user rating, available date. Filtering on these can provide recommendations within that data, such as movies that are available after a specific date, movies rated over 3 stars, movies from the 1990s etc

User Demographic ranges - you may want to recommend content to specific age demographics, for this you can create a filter that is specific to a age range like over 18, over 18 AND under 30, etc).

Lets look at the item metadata and user interactions, so we can get an idea what type of filters we can create.

In [13]:
# Create a dataframe for the items by reading in the correct source CSV
items_meta_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)

# Render some sample data
items_meta_df.head(10)

,GENRE,YEAR,CREATION_TIMESTAMP
ITEM_ID,,,
1,Adventure|Animation|Children|Comedy|Fantasy,1995,0
2,Adventure|Children|Fantasy,1995,0
3,Comedy|Romance,1995,0
4,Comedy|Drama|Romance,1995,0
5,Comedy,1995,0
6,Action|Crime|Thriller,1995,0
7,Comedy|Romance,1995,0
8,Adventure|Children,1995,0
9,Action,1995,0


Since there are alot of genres to filter on, we will create a filter, using the dynamic variable $GENRE, this will allow us to pass in the variable at runtime rather than create a static filter for each genre.

In [14]:
creategenrefilter_response = personalize.create_filter(name='Genre',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ($GENRE)'
    )

In [15]:
genre_filter_arn = creategenrefilter_response['filterArn']

Since we now have the year available in our item metadata, lets create a decade filter to recommend only movies releaseed in a given decade. A soft limit of Personalize at this time is 10 total filters, so we will create 7 decade filters for this workshop, leaving room for additional static and dynamic filters

Now create a list for the metadata decade filters and then create the actual filters with the cells below. Note this will take a few minutes to complete.

In [16]:
decades_to_filter = [1950,1960,1970,1980,1990,2000,2010]

In [17]:
# Create a list for the filters:
decade_filter_arns = []

In [18]:
# Iterate through Decades
for decade in decades_to_filter:
    # Start by creating a filter
    current_decade = str(decade)
    next_decade = str(decade + 10)
    try:
        createfilter_response = personalize.create_filter(
            name=current_decade + "s",
            datasetGroupArn=dataset_group_arn,
            filterExpression='INCLUDE ItemID WHERE Items.YEAR >= '+ current_decade +' AND Items.YEAR < '+ next_decade +''
    )
        # Add the ARN to the list
        decade_filter_arns.append(createfilter_response['filterArn'])
        print("Creating: " + createfilter_response['filterArn'])
    
    # If this fails, wait a bit
    except ClientError as error:
        # Here we only care about raising if it isnt the throttling issue
        if error.response['Error']['Code'] != 'LimitExceededException':
            print(error)
        else:    
            time.sleep(120)
            createfilter_response = personalize.create_filter(
                name=current_decade + "s",
                datasetGroupArn=dataset_group_arn,
                filterExpression='INCLUDE ItemID WHERE Items.YEAR >= '+ current_decade +' AND Items.YEAR < '+ next_decade +''
    )
            # Add the ARN to the list
            decade_filter_arns.append(createfilter_response['filterArn'])
            print("Creating: " + createfilter_response['filterArn'])

Creating: arn:aws:personalize:us-east-1:832194813872:filter/1950s
Creating: arn:aws:personalize:us-east-1:832194813872:filter/1960s
Creating: arn:aws:personalize:us-east-1:832194813872:filter/1970s
Creating: arn:aws:personalize:us-east-1:832194813872:filter/1980s
Creating: arn:aws:personalize:us-east-1:832194813872:filter/1990s
Creating: arn:aws:personalize:us-east-1:832194813872:filter/2000s
Creating: arn:aws:personalize:us-east-1:832194813872:filter/2010s


Lets look at the type of event data we can create filters for

In [19]:
# Create a dataframe for the interactions by reading in the correct source CSV
interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# Render some sample data
interactions_df.head(10)

,ITEM_ID,TIMESTAMP,EVENT_TYPE
USER_ID,,,
429,222,828124615,watch
429,227,828124615,click
429,595,828124615,watch
429,592,828124615,watch
429,590,828124615,watch
429,434,828124615,watch
429,421,828124615,watch
429,225,828124615,click
429,343,828124615,click


Since we may want to recommend items already watched, or not watchet yet, lets also create 2 event filters for watched and unwatched content. 

In [20]:
createwatchedfilter_response = personalize.create_filter(name='watched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )

createunwatchedfilter_response = personalize.create_filter(name='unwatched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='EXCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )

Before we are done we will want to add those filters to a list as well so they can be used later.

In [21]:
interaction_filter_arns = [createwatchedfilter_response['filterArn'], createunwatchedfilter_response['filterArn']]

In [22]:
%store sims_campaign_arn
%store userpersonalization_campaign_arn
%store rerank_campaign_arn
%store decade_filter_arns
%store genre_filter_arn
%store interaction_filter_arns



Stored 'sims_campaign_arn' (str)
Stored 'userpersonalization_campaign_arn' (str)
Stored 'rerank_campaign_arn' (str)
Stored 'decade_filter_arns' (list)
Stored 'genre_filter_arn' (str)
Stored 'interaction_filter_arns' (list)


You're all set to move on to the last exploratory notebook: `05_Interacting_with_Campaigns_and_Filters.ipynb`. Open it from the browser and you can start interacting with the Campaigns and gettign recommendations!